In [39]:
import tweepy
from textblob import TextBlob
import re
import pandas as pd

In [40]:
consumer_key= '1BMPtqx17ZSLIty2EBtNjkf6w'
consumer_secret= '4NWCsNou9NbSe9YIFIsUa2TvFK4UAbE6wNzrhB9cPPFbTPqs74'
access_token= '884241594018275328-i5mg2FHroHcYFB9elnTioaPYDANaY0E'
access_token_secret= 'rDvQlRB0SK3S90mHz9ebuSlUpEsZaOK5Q9LBnmecMBWNB'

In [41]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [42]:
def polarity(analysis):
	threshold = 0
	if analysis == threshold:
		return 'Neutral'
	elif analysis > threshold:
		return 'Positive'
	else:
		return 'Negative'

In [43]:
def clean_tweet(tweet):
	cleanedtweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+'')", " ", tweet).split())
	return cleanedtweet


In [38]:
query = 'Harbolnas'
max_tweets = 1000
public_tweets = [status for status in tweepy.Cursor(api.search, q=query).items(max_tweets)]
tweet_txt = []
tweet_stm = []
searched_tweets = []
last_id = -1
while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        new_tweets = api.search(q=query, count=count, max_id=str(last_id - 1))
        if not new_tweets:
            break
        searched_tweets.extend(new_tweets)
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # depending on TweepError.code, one may want to retry or wait
        # to keep things simple, we will give up on an error
        break


In [44]:
for tweet in public_tweets:
    cleanedtext = clean_tweet(tweet.text)
    tweet_txt.append(cleanedtext)
    analysis = TextBlob(cleanedtext)
    tweet_stm.append(polarity(analysis.sentiment.polarity))

In [45]:
download_dir = "hasilCrawling.csv"

In [46]:
df = pd.DataFrame(
    {
        "text" : tweet_txt,
        "polarity" : tweet_stm
    }
)
df.to_csv(download_dir)

#Step 7 Membaca data dari file CSV
colnames = ['text','polarity']
data = pd.read_csv('hasilCrawling.csv')

data_info=data.polarity.value_counts()
l = len(data)
print ("Neutral :"+str( (data_info['Neutral'] / l) * 100) + " %" )
print ("Positive :"+str( (data_info['Positive'] / l) * 100) + " %" )
print ("Negative :"+str( (data_info['Negative'] / l) * 100) + " %" )

Neutral :96.89999999999999 %
Positive :2.5 %
Negative :0.6 %
